In [37]:
import importlib

import pdc_functions.assumption_utils as assumptions
import pdc_functions.data_utils as data
import pdc_functions.explore_utils as explore
import pdc_functions.pdc_1and2_utils as pdc
import pdc_functions.stats_utils as stats
from dotenv import load_dotenv
import pdc_functions.class_utils as class_utils




importlib.reload(stats)



from phmlondon.snow_utils import SnowflakeConnection


In [2]:
load_dotenv()

snowsesh = SnowflakeConnection()
snowsesh.use_database("INTELLIGENCE_DEV")
snowsesh.use_schema("AI_CENTRE_FEATURE_STORE")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/8076439c-5b1f-4e15-91ea-a0bff0b3bf16/saml2?SAMLRequest=nZJBj9owEIX%2FSuSekzghyYIFrNKlCCR2iyC0FTfHccAisanH2bD99XUCSNvD7qE3y34z3%2FO8GT9e6sp55RqEkhMUeBg5XDJVCHmYoF02d4fIAUNlQSsl%2BQS9cUCP0zHQujqTtDFHueG%2FGw7GsY0kkO5hghotiaIggEhacyCGkW36vCKhhwkF4NpYHLqVFCAs62jMmfh%2B27ZeO%2FCUPvghxtjHI9%2BqOskX9A5x%2Fpxx1soopqp7ycX%2B6QNE4OOoQ1iFJaxvhV%2BFvI7gM0p%2BFQFZZNnaXX%2FfZshJ7797UhKamust16%2BC8d1mdTUA1sH2W5wEw8RrTi4oO0OP%2Fmk090CqtqzoiTNVnxtjW3v25Je88Ct1EHZgy9kEnU%2BiyH%2BynTldEjimKZ1v2P7XarNo2PPg8rDOZwXbY52N8jwyixNDzo97vGEX7xKg4UvZhWrsFQ5jF0duGGdBSOKEhLE3jEZ75MxsqEJS01fenfc%2BvFowrUCVRslKSN67HOKHJBqMmBvnQelGPIjdUcCpS3Feljgf5GWQ%2BF10IbquD%2BmN6Ol%2FDWXsv29xW8cXm9BytlaVYG%2FOXOmamo8DDLygvxGFW%2FZSwmsqqrQoNAewQVaVap80p8

In [3]:
df = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.compliance_cohort_1yr_top7")



Retrieved 8074910 rows from cohort table: intelligence_dev.ai_centre_dev.compliance_cohort_1yr_top7


In [4]:
medication_table = class_utils.MedicationTable(df)

Run checks on the dataframe using the Medication Table Class functions

In [5]:
medication_table.validate_columns()  # Validate columns
medication_table.normalise_missing_values()  # Standardise representation of missing data
medication_table.convert_dates()  # Convert date columns
medication_table.validate_data_types()  # Validate data types
medication_table.clean_dose()  # Clean and interpret the 'dose' column
medication_table.calculate_covered_days()

print("All validations passed successfully!")

Invalid rows (dose or tablets_per_day issue):
Empty DataFrame
Columns: [dose, tablets_per_day]
Index: []
All validations passed successfully!


In [6]:
pdc_data = pdc.compute_pdc_overall(medication_table.df)


In [7]:
print(pdc_data.head())



     order_id  person_id  medication_statement_id                drug_name  \
0  1102325986     456338                361607272  Metformin hydrochloride   
1  3262543494     456338                361607272  Metformin hydrochloride   
2  3365121928     456338                361607272  Metformin hydrochloride   
3  3398132582     456338                361607272  Metformin hydrochloride   
4  3428901099     456338                361607272  Metformin hydrochloride   

                          dose  quantity_value quantity_unit  \
0  Two To Be Taken Twice A Day           224.0        tablet   
1  Two To Be Taken Twice A Day           224.0        tablet   
2  Two To Be Taken Twice A Day           224.0        tablet   
3  Two To Be Taken Twice A Day           224.0        tablet   
4  Two To Be Taken Twice A Day           224.0        tablet   

          drug_description       class  core_concept_id  ...  \
0  Metformin 500mg tablets  Biguanides          3136770  ...   
1  Metformin 500mg

In [10]:
pdc_data_interval = pdc.compute_pdc_intervals(pdc_data)

In [15]:
print(pdc_data_interval.columns)

Index(['order_id', 'person_id', 'medication_statement_id', 'drug_name', 'dose',
       'quantity_value', 'quantity_unit', 'drug_description', 'class',
       'core_concept_id', 'order_date', 'duration_days', 'days_to_next_order',
       'statement_date', 'statement_enddate', 'medication_compliance',
       'compliance_date', 'tablets_per_day', 'calculated_duration',
       'order_enddate', 'covered_days', 'overall_inclusive_pdc',
       'overall_exclusive_pdc', 'total_covered_days', 'total_exposure_days',
       'interval', 'pre_inclusive_pdc', 'pre_exclusive_pdc',
       'total_pre_covered_days', 'total_pre_exposure_days',
       'post_inclusive_pdc', 'post_exclusive_pdc', 'total_post_covered_days',
       'total_post_exposure_days'],
      dtype='object')


In [19]:
cohort_table = data.add_demographic_data(snowsesh, pdc_data_interval)


Merged data: 13960669 rows with columns: ['order_id', 'person_id', 'medication_statement_id', 'drug_name', 'dose', 'quantity_value', 'quantity_unit', 'drug_description', 'class', 'core_concept_id', 'order_date', 'duration_days', 'days_to_next_order', 'statement_date', 'statement_enddate', 'medication_compliance', 'compliance_date', 'tablets_per_day', 'calculated_duration', 'order_enddate', 'covered_days', 'overall_inclusive_pdc', 'overall_exclusive_pdc', 'total_covered_days', 'total_exposure_days', 'interval', 'pre_inclusive_pdc', 'pre_exclusive_pdc', 'total_pre_covered_days', 'total_pre_exposure_days', 'post_inclusive_pdc', 'post_exclusive_pdc', 'total_post_covered_days', 'total_post_exposure_days', 'date_of_birth', 'gender', 'ethnicity', 'imd']


In [20]:
cohort_table_agg = data.agg_data_person_drug(cohort_table)

In [22]:
table_one = explore.get_table_one(cohort_table_agg,"medication_compliance")

                                                                             Grouped by medication_compliance                                                       
                                                                                                      Missing         Overall            good           poor P-Value
n                                                                                                                      300843          269223          31620        
gender, n (%)                       Female                                                                      152959 (50.8)   137852 (51.2)   15107 (47.8)  <0.001
                                    Male                                                                        147850 (49.1)   131337 (48.8)   16513 (52.2)        
                                    None                                                                              3 (0.0)         3 (0.0)                       
          

# note about the missingness
missingness in pre and post pdc is due to participants being included but not having any orders before or after the compliance status date.

In [23]:
missing_pdc = cohort_table_agg[cohort_table_agg['overall_inclusive_pdc'].isna()]
missing_pdc['total_exposed_days'].value_counts(dropna=False)

total_exposed_days
0    58
Name: count, dtype: int64

In [36]:
covariates = ['gender', 'ethnicity', 'imd', 'drug_class', 'age_at_start']
stats.fit_and_save_models_for_pdc(cohort_table_agg, outcome_col='medication_compliance',
                            pdc_cols=['overall_inclusive_pdc',
                                      'overall_exclusive_pdc',
                                      'pre_inclusive_pdc',
                                      'pre_exclusive_pdc',
                                      'post_inclusive_pdc',
                                      'post_exclusive_pdc'],
                            covariates=covariates,
                            cluster_col='person_id')

Results saved to unadjusted_results.csv


/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)
/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/meds_env/lib/python3.11/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwar

Results saved to adjusted_results.csv
Results saved to multilevel_unadjusted_results.csv
Results saved to multilevel_adjusted_results.csv


In [ ]:
stats.unadjusted_logr(cohort_table_agg)


In [ ]:
# List of covariates to include in model

stats.adjusted_logr(cohort_table_agg, covariates=covariates)


# Multilevel Logistic regression

In [ ]:
stats.multilevel_unadjusted_logr(cohort_table_agg)

In [ ]:
covariates = ['gender', 'ethnicity', 'imd', 'drug_class', 'age_at_start', 'total_exposed_days']
multiadj_model, multiadj_or = stats.multilevel_adjusted_logr(cohort_table_agg, covariates=covariates)

In [ ]:
multiadj_or

# check assumptions

In [ ]:

# 1. Linearity check
assumptions.check_linearity_log_odds(cohort_table_agg, outcome_col='medication_compliance', continuous_predictors=['dynamic_pdc', 'age_at_start'])

In [ ]:
# 2. Multicollinearity
assumptions.check_vif(cohort_table_agg, predictors=['dynamic_pdc', 'age_at_start', 'total_exposed_days'])

In [ ]:
# 3. Within-cluster variation
assumptions.check_within_cluster_variation(cohort_table_agg, cluster_col='person_id', outcome_col='medication_compliance')

In [ ]:
# 4. ICC
assumptions.estimate_icc(cohort_table_agg, outcome_col='medication_compliance', cluster_col='person_id')

In [ ]:
# 5. Residuals plot
# Drop rows with missing values
cohort_table_agg_clean = cohort_table_agg.dropna()

# Map 'good' to 0 and 'poor' to 1 in the outcome column (this step is applied to the dataframe directly)
cohort_table_agg_clean['medication_compliance'] = cohort_table_agg_clean['medication_compliance'].map({'good': 0, 'poor': 1})

# Now call the function with the cleaned dataframe and outcome column name as a string
assumptions.plot_residuals_vs_fitted(multiadj_model, cohort_table_agg_clean, outcome_col='medication_compliance')